<a href="https://colab.research.google.com/github/blasmod/03MAIR----Algoritmos-de-Optimizacion---2020/blob/master/AG3/Blas_Rodriguez_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Actividad Guiada AG3 Algoritmos de Optimización

Blas Rodriguez Lerena

https://colab.research.google.com/drive/1MGkboPu7bvsXbNsZj0NPHpFG-EiqW4W4?usp=sharing

In [11]:
!pip install request
!pip install tsplib95

In [12]:
import urllib. request

file = "swiss42.tsp"

urllib.request.urlretrieve('http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp',file)

('swiss42.tsp', <http.client.HTTPMessage at 0x7f92fa905b70>)

In [13]:
import tsplib95
import random
import math
from math import e,inf
import copy

In [40]:
problem = tsplib95.load(file)   #problem contiene clases como funciones

Nodos = list(problem.get_nodes())

Aristas = list(problem.get_edges())

In [41]:
#Genera solución aleatoria con comienzo en el nodo 0
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  
  for n in Nodos[1:]:
    
    #print (solucion)
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion


 # crear_solucion(Nodos)

#Devuelve distancia entre nodos
def distancia(a,b,problem):
  return problem.get_weight(a,b)

#Devuelve distancia total de una trayectoria / solución
def distancia_total(solucion,problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)

  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)



In [42]:
#Realizamos el cálculo de la distancia en primer lugar con la búsqueda
#aleatoria según lo visto en teoría
def busqueda_aleatoria(problem,N):
  mejor_solucion = []
  mejor_distancia = inf #infinito

  for i in range (N):
    solucion = crear_solucion(Nodos)
    distancia = distancia_total(solucion,problem) 

    if distancia < mejor_distancia:
      mejor_solucion = solucion
      mejor_distancia = distancia

  return mejor_solucion
  
s=busqueda_aleatoria(problem,100)     
print(distancia_total(s,problem),s)   
            

4262 [0, 32, 26, 28, 25, 11, 36, 6, 9, 10, 5, 4, 16, 18, 24, 22, 17, 15, 14, 37, 38, 27, 19, 1, 3, 12, 31, 21, 39, 7, 35, 33, 2, 20, 29, 13, 40, 41, 23, 34, 8, 30]


In [26]:
#Búsqueda Local. Generador de vecindad
#Generador de soluciones vecinas: 2 - opt 
#(intercambiar nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones 
def genera_vecina(solucion,problem):
  mejor_solucion = []
  mejor_distancia = inf #infinito
  for i in range (1,len(solucion)-1):
    for j in range(i+1,len(solucion)):
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:] 
      #Divido solución de referencia en tres partes
      distancia_vecina = distancia_total(vecina,problem) 
      if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina
  return mejor_solucion

s1 = crear_solucion(Nodos)
s2 = genera_vecina(s1,problem)
#print(s1)
print(distancia_total(s1,problem),s1)
print(distancia_total(s2,problem),s2)

4789 [0, 40, 32, 21, 22, 9, 8, 14, 27, 17, 7, 18, 23, 2, 13, 4, 16, 30, 35, 6, 3, 29, 33, 20, 12, 10, 34, 19, 41, 39, 31, 28, 1, 25, 11, 38, 37, 26, 24, 36, 5, 15]
4255 [0, 40, 24, 21, 22, 9, 8, 14, 27, 17, 7, 18, 23, 2, 13, 4, 16, 30, 35, 6, 3, 29, 33, 20, 12, 10, 34, 19, 41, 39, 31, 28, 1, 25, 11, 38, 37, 26, 32, 36, 5, 15]


In [27]:
#Función para obtener la distancia según el algoritmo de búsqueda local visto
def busqueda_local(problem):
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = 10e100

  iteracion = 0
  while(1):
    iteracion+=1
    vecina = genera_vecina(solucion_referencia,problem)
    distancia_vecina = distancia_total(vecina,problem) 
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
    else:
      print('En la iteracion ' + str(iteracion)+
            ' encontramos la solucion:'+str(mejor_distancia))
      print("Distancia :",mejor_distancia)
      print(mejor_solucion)
      return mejor_solucion
    solucion_referencia = vecina
sol = busqueda_local(problem)


En la iteracion 32 encontramos la solucion:1615
Distancia : 1615
[0, 3, 6, 1, 17, 31, 36, 35, 20, 33, 34, 32, 14, 16, 15, 37, 7, 18, 12, 11, 25, 41, 23, 40, 24, 21, 39, 22, 38, 9, 8, 10, 13, 19, 5, 26, 4, 29, 30, 28, 2, 27]


In [43]:
def genera_vecina_aleatorio(solucion):
#Generador de una solución vecina 2-opt aleatoria (intercambiar 2 nodos)

#Se eligen dos nodos aleatoriamente y se intercambian
  i,j = sorted(random.sample( range(1,len(solucion)) , 2))

  return solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

def bajar_temperatura(T):
  return T*.999 #Descenso exponencial

def probabilidad(T,d):
  if random.random() <= math.exp( -1*d / T)  :
    return True
  else:
    return False


In [38]:
def recocido_simulado(problem, TEMPERATURA=1000):
  #problem = datos del problema

  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)

  mejor_solucion = []
  mejor_distancia = 10e100
  N = 0
  while TEMPERATURA > .0001:
    N+=1
    #Genera una solución vecina
    vecina = genera_vecina_aleatorio(solucion_referencia)

    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)

    #Si es la mejor solución de todas se guarda
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
  
    #Si la nueva vecina es mejor se cambia
    # Si es peor se cambia según una probabilidad 
    #que depende de T y delta(distancia_referencia - distancia_vecina)
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) ):
      solucion_referencia = copy.deepcopy(vecina)
      distancia_referencia = distancia_vecina

    #Bajamos la temperatura
    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solución encontrada es ", end="")
  print(mejor_solucion)
  print("con una distancia total de ", end="")
  print(mejor_distancia)
  return mejor_solucion



In [44]:
sol = busqueda_local(problem)  
sol = recocido_simulado(problem, 1000000)

En la iteracion 36 encontramos la solucion:1802
Distancia : 1802
[0, 27, 29, 39, 22, 30, 28, 2, 4, 18, 12, 11, 25, 41, 23, 9, 38, 34, 33, 20, 32, 6, 1, 31, 35, 36, 17, 37, 15, 16, 14, 7, 19, 13, 5, 26, 10, 40, 24, 21, 8, 3]
La mejor solución encontrada es [0, 37, 15, 16, 14, 4, 3, 2, 27, 32, 20, 33, 34, 9, 8, 10, 18, 6, 1, 7, 17, 31, 35, 36, 19, 13, 5, 26, 12, 11, 25, 41, 23, 40, 24, 21, 39, 22, 38, 30, 29, 28]
con una distancia total de 1653


##Algoritmo de colonia de hormigas

La función Add_Nodo selecciona al azar un nodo con probabilidad uniforme.
Para ser mas eficiente debería seleccionar el próximo nodo siguiendo la probabilidad correspondiente a la ecuación:

$p^k_{ij}(t) = \frac{[\tau_{ij}(t)]^\alpha[\nu_{ij}]^\beta}{\sum_{l\in J^k_i} [\tau_{il}(t)]^\alpha[\nu_{il}]^\beta}$, si $j \in J^k_i$

$p^k_{ij}(t) = 0$, si $j \notin J^k_i$

In [45]:
#Funciones Auxiliares
def Add_Nodo(problem, H ,T ) :
  #Mejora:Establecer una funcion de probabilidad para 
  # añadir un nuevo nodo dependiendo de los nodos mas cercanos y 
  #de las feromonas depositadas
  Nodos = list(problem.get_nodes())
  return random.choice(   list(set(range(1,len(Nodos))) - set(H) )  )


def Incrementa_Feromona(problem, T, H ) :
  #Incrementa segun la calidad de la solución. 
  #Añadir una cantidad inversamente proporcional a la distancia total 
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T

def Evaporar_Feromonas(T ):
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
  #Mejora:Podemos elegir diferentes funciones de evaporación 
  #dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
  T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(Nodos)) ]
       for j in range(len(Nodos))]
  return T

In [46]:
#Método Constructivo. Colonia de Hormigas
def hormigas(problem, N) :
  #problem = datos del problema
  #N = Número de agentes(hormigas)
    
  #Nodos
  Nodos = list(problem.get_nodes())
  #Aristas
  Aristas = list(problem.get_edges()) 
  
  #Inicializa las aristas con una cantidad inicial de feromonas:1
  #Mejora: inicializar con valores diferentes dependiendo diferentes criterios
  T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
  
  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]
  
  #Recorre cada agente construyendo la solución
  for h in range(N) :
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1) :
      
      #Elige el siguiente nodo
      Nuevo_Nodo = Add_Nodo(problem, Hormiga[h] ,T )
      Hormiga[h].append(Nuevo_Nodo)     
    
    #Incrementa feromonas en esa arista 
    T = Incrementa_Feromona(problem, T, Hormiga[h] )
    #print("Feromonas(1)", T)
      
    #Evapora Feromonas  
    T = Evaporar_Feromonas(T)
    #print("Feromonas(2)", T)

    #Seleccionamos el mejor agente
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N) :
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia =distancia_actual
  
  
  print(mejor_solucion)
  print(mejor_distancia)
  
  
hormigas(problem, 1000)

[0, 4, 2, 6, 35, 20, 39, 11, 41, 28, 38, 13, 12, 16, 23, 18, 1, 7, 17, 32, 26, 29, 25, 21, 22, 30, 15, 19, 14, 33, 31, 3, 37, 36, 34, 27, 5, 24, 40, 9, 10, 8]
3782
